In [1]:
# !wget https://storage.googleapis.com/allennlp-data-bucket/nllb/eng_Latn-zho_Hant.gz
# !gzip -d eng_Latn-zho_Hant.gz
# !wc -l /home/husein/ssd3/multilang/eng_Latn-zho_Hant

In [2]:
# !head -n 2 /home/husein/ssd3/multilang/eng_Latn-zho_Hant

In [3]:
import json
from tqdm import tqdm
import re

def language_detection_textcleaning(string):
    string = re.sub(
        'http\\S+|www.\\S+',
        '',
        ' '.join(
            [i for i in string.split() if i.find('#') < 0 and i.find('@') < 0]
        ),
    )

    chars = ',.()!:\'"/;=-'
    for c in chars:
        string = string.replace(c, f' {c} ')
    string = string.replace('\n', '').replace('\t', '')

    string = re.sub(
        '[0-9!@#$%^&*()_\\-+{}|\\~`\'";:?/.>,<]', ' ', string, flags=re.UNICODE
    )
    string = re.sub(r'[ ]+', ' ', string).strip()

    return string.lower()

In [4]:
with open('standard-mandarin.jsonl', 'w') as fopen_l:
    with open('/home/husein/ssd3/multilang/eng_Latn-zho_Hant') as fopen:
        for row in tqdm(fopen):
            splitted = row.split('\t')
            if float(splitted[2]) >= 1.07:
                fopen_l.write(f'{json.dumps(splitted[1])}\n')
            else:
                break

2265456it [00:04, 464255.71it/s]


In [5]:
from bs4 import BeautifulSoup
import malaya

In [6]:
language_detection = malaya.language_detection.fasttext(model = 'mesolitica/fasttext-language-detection-v1')

In [ ]:
# local_mandarin = []
# with open('/home/husein/ssd3/cn-cari-com-my/everything.jsonl') as fopen:
#     for l in tqdm(fopen):
#         data = json.loads(l)
#         soup = BeautifulSoup(data, "lxml")
#         texts = [d.text for d in soup.find_all('div')]
#         texts = [language_detection_textcleaning(t) for t in texts if t and len(t)]
#         texts = [t for t in texts if len(t) and language_detection.predict([t])[0] in {'other'}]
#         local_mandarin.extend(texts)

In [12]:
!wc -l /home/husein/ssd3/c-cari-com-my/everything.jsonl

14538890 /home/husein/ssd3/c-cari-com-my/everything.jsonl


In [14]:
with open('local-mandarin.jsonl', 'w') as fopen_l:
    with open('/home/husein/ssd3/c-cari-com-my/everything.jsonl') as fopen:
        for l in tqdm(fopen):
            data = json.loads(l)
            if language_detection.predict([data])[0] not in {'other'}:
                continue
            fopen_l.write(f'{json.dumps(data)}\n')

14538890it [05:34, 43408.42it/s]


In [15]:
!wc -l local-mandarin.jsonl

8502399 local-mandarin.jsonl
